In [10]:
import numpy as np
import pandas as pd

path = "./ninapro_db1/Ninapro_DB1.csv"
df = pd.read_csv(path)
df = df.iloc[:2000000]
emg_cols = [c for c in df.columns if "emg" in c.lower()]
stim = df["stimulus"]
stim = stim.squeeze()
df = df[emg_cols]

df.shape, stim.shape

((2000000, 10), (2000000,))

In [27]:
from scipy import signal

def emg_filter_pipeline(x, fs, band=(20, 90), notch=60, smooth_lp=5.0, order=4, rectify=False):
    x = np.asarray(x, dtype=float)
    nyq = fs * 0.5

    def _filtfilt(b, a, sig):
        return signal.filtfilt(b, a, sig, axis=0, method="gust")

    # Bandpass
    low, high = band
    high = min(high, nyq * 0.999)               # keep strictly < Nyquist
    assert 0 < low < high < nyq, "Invalid band edges for given fs."
    b_bp, a_bp = signal.butter(order, [low/nyq, high/nyq], btype="band")
    y = _filtfilt(b_bp, a_bp, x)

    # Notch 60Hz
    if notch is not None:
        w0 = notch / nyq
        # Quality factor: higher Q = narrower notch (30–50 works well for EMG mains hum)
        b_notch, a_notch = signal.iirnotch(w0, Q=30)
        y = _filtfilt(b_notch, a_notch, y)

    # Rectifying
    if rectify:
        y = np.abs(y)

    # Smoothing
    if smooth_lp is not None:
        cutoff = min(smooth_lp, nyq * 0.999)
        b_lp, a_lp = signal.butter(order, cutoff/nyq, btype="low")
        y = _filtfilt(b_lp, a_lp, y)

    return y

df = emg_filter_pipeline(
    df, 
    fs = 100,
    band = (5, 45),
    notch = None,
    smooth_lp = None,
    order = 4,
    rectify = False
)


In [28]:
from scipy.stats import mode

window_size = 200

def window(df, win_size, stride = 1):
    windows = []
    for start in range(0, df.shape[0] - win_size, stride):
        windows.append(df[start:start+win_size])
    return np.stack(windows, axis=0)

data = window(df, win_size=window_size, stride = 10)
y = window(stim, win_size=window_size, stride = 10)
y = mode(y, axis=1)[0].squeeze()


In [29]:
data.shape, y.shape

((199980, 200, 10), (199980,))

In [30]:
import torch

x = torch.tensor(data).float()
x = x.permute(0, 2, 1) 


In [31]:
import torch.nn as nn
import torch.nn.functional as F

class CNN_LSTM(nn.Module):
    def __init__(self, in_channels, hidden_size, num_layers, n_classes):
        super(CNN_LSTM, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, 32, kernel_size=7, padding=3)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.bn2 = nn.BatchNorm1d(64)
        # self.relu = F.relu()
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)

        self.hidden_dim = 64
        self.lstm = nn.LSTM(input_size = self.hidden_dim, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout=0.3 if num_layers > 1 else 0)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_size, 64)
        self.fc2 = nn.Linear(64, n_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        # x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        # x = self.relu(x)
        x = self.pool(x)

        x = x.permute(0, 2, 1)

        lstm_out, (hn, cn) = self.lstm(x)
        out = lstm_out[:, -1, :]
        out = self.dropout(out)
        x = F.relu(self.fc(out))
        x = self.dropout(x)
        logits = self.fc2(x)
        return logits

In [33]:
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# import numpy as np

class EMGDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        x = self.X[index]
        y = self.y[index]
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.long)
        return x, y

X_train, X_temp, y_train, y_temp = train_test_split(x, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

mu = X_train.mean(axis=(0, 2), keepdims = True)
sig = X_train.std(axis=(0, 2), keepdims = True) + 1e-8

X_train = (X_train - mu) / sig
X_val = (X_val - mu) / sig
X_test = (X_test - mu) / sig


train = EMGDataset(X_train, y_train)
val = EMGDataset(X_val, y_val)

train_loader = DataLoader(train, batch_size=64, shuffle=True)
val_loader = DataLoader(val, batch_size=64, shuffle=False)

In [36]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
n_channels = 10
n_classes = 24
model = CNN_LSTM(in_channels=n_channels, hidden_size=64, num_layers=2, n_classes=n_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)


epochs = 100

train_acc_history = []
val_acc_history = []

for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for xb, yb in train_loader:
        xb = xb.to(device)
        yb = yb.to(device)

        optimizer.zero_grad()
        logits = model(xb)
        loss =criterion(logits, yb)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * xb.size(0)
        preds = logits.argmax(dim=1)
        correct += (preds == yb).sum().item()
        total += xb.size(0)

    train_loss = total_loss / total
    train_acc = correct / total

    train_acc_history.append(train_acc)

    model.eval()
    val_total = 0
    val_correct = 0
    val_loss = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb = xb.to(device)
            yb = yb.to(device)
            logits = model(xb)
            # loss =criterion(logits, yb)
            preds = logits.argmax(dim=1)
            val_correct += (preds == yb).sum().item()
            val_total += xb.size(0)

    val_acc = val_correct/val_total

    val_acc_history.append(val_acc)



    print(f"Epoch {epoch+1}: train_loss={train_loss:.4f}, train_acc={train_acc:.3f}, val_acc={val_acc:.3f}")


/var/folders/c1/t6c91th93rg_t8cmmzfjwp840000gn/T/ipykernel_98741/4236329272.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32)


Epoch 1: train_loss=2.1711, train_acc=0.427, val_acc=0.439
Epoch 2: train_loss=2.1275, train_acc=0.432, val_acc=0.403
Epoch 3: train_loss=2.2216, train_acc=0.425, val_acc=0.429
Epoch 4: train_loss=2.2166, train_acc=0.426, val_acc=0.426
Epoch 5: train_loss=2.1961, train_acc=0.427, val_acc=0.423
Epoch 6: train_loss=2.1787, train_acc=0.429, val_acc=0.053
Epoch 7: train_loss=2.1726, train_acc=0.429, val_acc=0.426
Epoch 8: train_loss=2.1695, train_acc=0.430, val_acc=0.416
Epoch 9: train_loss=2.1656, train_acc=0.431, val_acc=0.426
Epoch 10: train_loss=2.1543, train_acc=0.432, val_acc=0.431
Epoch 11: train_loss=2.1539, train_acc=0.433, val_acc=0.430
Epoch 12: train_loss=2.1539, train_acc=0.434, val_acc=0.430
Epoch 13: train_loss=2.1519, train_acc=0.432, val_acc=0.392
Epoch 14: train_loss=2.1510, train_acc=0.433, val_acc=0.434
Epoch 15: train_loss=2.1488, train_acc=0.432, val_acc=0.437
Epoch 16: train_loss=2.1520, train_acc=0.432, val_acc=0.434
Epoch 17: train_loss=2.1507, train_acc=0.433, val